# Lab 3: Singing a song
## Introduction
The song “12 Days of Christmas”, written around 1780, tells the tale of many gifts a person receives in the days leading up to Christmas (link to lyrics.

Note
You can watch a video of the 12 Days of Christmas at the Cambria Christmas Market.

These gifts repeat and compound; on the first day, the narrator receives

A partridge in a pear tree.
On the twelfth day, they receive

Twelve Drummers Drumming
Eleven Pipers Piping
Ten Lords a Leaping
Nine Ladies Waiting
Eight Maids a Milking
Seven Swans a Swimming
Six Geese a Laying
Five Golden Rings
Four Calling Birds
Three French Hens
Two Turtle Doves
And a Partridge in a Pear Tree
This week, your task will be to write functions that automatically sing this very repetitive song.

### Data set

Run the code provided to load in a data set called xmas that contains the crucial information about the gifts in the song. We will use this data set to test out our functions as we work on them.

In [50]:
import pandas as pd
import numpy as np
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

# Function 1: pluralize_gift()

The gifts are listed in singular: for example, on day five the narrator receives “five golden rings”, but the entry in the data set for the gift on day five simply says “ring”.

Hint 1: The gifts on days six and nine have unusual pluralization. You may assume that in other data sets, there will be no additional special cases besides these types.

Hint 2: The following small code snippets may be useful to you

In [111]:
def pluralize_gift(gift):
  """
  Returns plural of a noun
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version
  """


  if gift.find("oo") != -1:
    gift = gift.replace("oo", "ee")
  elif gift[-1] == "y":
    gift = gift.replace("y", "ies")
  else:
    gift = gift + "s"

  return gift

pluralize_gift = np.vectorize(pluralize_gift)



# Test your Function

In [112]:
# Should work
pluralize_gift("goose")

array('geese', dtype='<U5')

In [113]:
# Will work if your function is vectorized! 
pluralize_gift(xmas['Gift.Item'])

array(['partridges', 'doves', 'hens', 'birds', 'rings', 'geese', 'swans',
       'maids', 'ladies', 'lords', 'pipers', 'drummers'], dtype='<U10')

# Function 2: make_phrase()
Write a function called make_phrase() that takes as input the necessary information, and returns a phrase. For example,

make_phrase(num_word = "ten", 
            item = "lords", 
            verb = "a-leaping", 
            adjective = "", 
            location = "")
should return

"ten lords a-leaping"

In [118]:
def make_phrase(num, num_word, item, verb, adjective, location):
  """
  Returns plural of a noun
  
  Parameters
  ----------
  num: int
    A number
  num_word: str
    a number in words
  item: str
    a noun
  verb: str
    a verb
  adjective: str
    an adjective
  location: str
    a location
    
  Return
  ------
  str
    Combined phrase
  """
  
  ## Step 1: Replace NAs with blank strings
  verb = verb.fillna("")
  adjective = adjective.fillna("")
  location = location.fillna("")
  
  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  item = np.where(day > 1, pluralize_gift(item), item)
  
  ## Step 3: Figure out if a gift item starts with a vowel
  ## Step 4: For the first day, if the gift item starts with a vowel, 
  # replace the day with "an" and if the gift item does not start with a vowel, 
  # replace the day with "a" (e.g. a partridge in a pear tree). 
  # If it is not the first day, use just the number word (e.g. ten lords a leap)
  
  article = np.where((item[0] == "[aeiou]") & (num == 1), "an",
                      np.where(num == 1, "a", ""))
  num_word = np.where(num == 1, "", num_word)

  ## Step 5: Put all of the pieces together into one string and return!

  phrase = "{article} {num_word} {adjective} {item} {verb} {location}"

  return phrase

make_phrase = np.vectorize(make_phrase)


# Test Your Function
Make sure to try your function out on small examples and on the xmas data.

Then, use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day.

In [124]:
make_phrase(num = xmas["Day"], 
            num_word = xmas["Day.in.Words"],
            item = xmas["Gift.Item"],
            verb = xmas["Verb"],
            adjective = xmas["Adjective"],
            location = xmas["Location"])

AttributeError: 'float' object has no attribute 'fillna'